In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [2]:
# Modeli daha hızlı eğitmek için CPU yerine GPU seçtik. Bunun uygulanabilmesi için birkaç program indirilmesi gerekmektedir.
# Gerekenlere buradan ulaşabilirsiniz: https://www.tensorflow.org/install/gpu
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5



In [3]:
import pathlib
data_dir = pathlib.Path("./converted_images")
image_count = len(list(data_dir.glob('*/*.jpeg')))
print(image_count)
NoDR = list(data_dir.glob('0-NoDR/*'))
print(str(NoDR[0]))
#PIL.Image.open(str(NoDR[0]))

35126
converted_images\0-NoDR\10003_left.jpeg


In [4]:
batch_size = 32
img_height = 192
img_width = 256

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=142,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 35126 files belonging to 5 classes.
Using 24589 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=142,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 35126 files belonging to 5 classes.
Using 10537 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['0-NoDR', '1-Mild', '2-Moderate', '3-Severe', '4-ProliferativeDR']


In [8]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [9]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [10]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)), # Fotoğrafı yatay eksende rastgele bir şekilde döndürüyor.
    layers.RandomRotation(0.1), # Fotoğrafa rastgele bir rotasyon ekliyor.
    layers.RandomZoom(0.1), # Fotoğrafı rastgele boyutta büyütüp küçültüyor
  ]
)

In [11]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
num_classes = len(class_names) # Kaç tane sınıfımızın olduğunu tutuyoruz
model = Sequential([ # Sequential model en basit modeldir. Kolayca katman eklememizi sağlar
  data_augmentation, # Bir önceki kod parçacığındaki fotoğraf döndürme işlemi
  #layers.Rescaling(1./255), # Piksel değerlerini [0 255] aralığından [0 1] aralığına indirgiyoruz
  layers.Grayscale(output_channels=1)
  layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)), # Relu aktivasyonlu Conv2D katmanı
  layers.MaxPooling2D(), # MaxPooling2D ile veri havuzunun boyutunu küçültüyoruz.
  layers.Conv2D(32, 3, padding='same', activation='relu'), # Relu aktivasyonlu Conv2D katmanı
  layers.MaxPooling2D(), # MaxPooling2D ile veri havuzunun boyutunu küçültüyoruz.
  layers.Conv2D(64, 3, padding='same', activation='relu'), # Relu aktivasyonlu Conv2D katmanı
  layers.MaxPooling2D(), # MaxPooling2D ile veri havuzunun boyutunu küçültüyoruz.
  #layers.Conv2D(128, 3, padding='same', activation='relu'), # Relu aktivasyonlu Conv2D katmanı
  #layers.MaxPooling2D(), # MaxPooling2D ile veri havuzunun boyutunu küçültüyoruz.
  #layers.Dropout(0.2), # Aşırı uyumu engellemek için belli değerleri düşürüyoruz.
  layers.Flatten(), # Matris formundaki veriyi düzleştirmek için kullanılır
  layers.Dense(128, activation='sigmoid'), # Dense çoğu durumda çalışan standart bir katman türüdür. Yoğun bir katmanda, önceki katmandaki tüm düğümler mevcut katmandaki düğümlere bağlanır
  layers.Dense(num_classes) # Dense çoğu durumda çalışan standart bir katman türüdür. Yoğun bir katmanda, önceki katmandaki tüm düğümler mevcut katmandaki düğümlere bağlanır
])

In [13]:
opt = SGD(learning_rate=0.001)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [14]:
model.summary()

In [15]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy'] # Eğitim doğruluğunun değerlerini alıyoruz
val_acc = history.history['val_accuracy'] # Doğrulama doğruluğunun değerlerini alıyoruz

loss = history.history['loss'] # Eğitim kayıp verisini alıyoruz
val_loss = history.history['val_loss'] # Doğrulama kayıp verisini alıyoruz

epochs_range = range(epochs) # 0'dan [paket sayısı - 1]'e kadar bir dizi oluşturuyoruz [0, 1, ..., epochs - 1]

plt.figure(figsize=(8, 8)) # 8'e 8 boyutlu bir figür oluşturuyoruz
plt.subplot(1, 2, 1) # Doğruluk verisini içerek ilk parçayı ayırıyoruz
plt.plot(epochs_range, acc, label='Training Accuracy') # Eğitim doğruluğunun grafiğini çiziyoruz
plt.plot(epochs_range, val_acc, label='Validation Accuracy') # Doğrulama doğruluğunun grafiğini çiziyoruz
plt.legend(loc='lower right') # Legend yazısını sağ alta alıyoruz
plt.title('Training and Validation Accuracy') # Başlığı ayarlıyoruz

plt.subplot(1, 2, 2) # Kayıp verisini içeren ikinci parçayı ayırıyoruz
plt.plot(epochs_range, loss, label='Training Loss') # Eğitim kaybının grafiğini çiziyoruz
plt.plot(epochs_range, val_loss, label='Validation Loss') # Doğruluk kaybının grafiğini çiziyoruz
plt.legend(loc='upper right') # Legend yazısını sağ üste alıyoruz
plt.title('Training and Validation Loss') # Başlığı ayarlıyoruz
plt.show() # Grafiklerimizi gösteriyoruz